In [100]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2
from skimage.color import rgb2gray
from skimage import io
from skimage.measure import label, regionprops

class ImageAnalysis:
    
    """
    Provides methods for organizing and analyzing image data from various directory structures within a project folder.
    This class initializes by reading the project folder, creating a DataFrame that lists each directory and its path,
    and allows for further expansion to include specific image analysis metadata.

    Parameters:
    - project_folder (str): The path to the project folder containing various image data directories.

    Attributes:
    - project_folder (str): Stores the provided path to the project folder.
    - directory_df (DataFrame): A pandas DataFrame listing directories within the project folder and various attributes
      related to image analysis.

    Methods:

    __init__(self, project_folder)
        Constructor for the ImageAnalysis class. Initializes the project folder and sets up the initial directory
        DataFrame by listing all directories within the provided project folder.

    initialize_directory_df(self)
        Scans the project folder to list all subdirectories and creates a DataFrame with the directory names and their
        respective paths.

        Returns:
        - DataFrame: A pandas DataFrame with columns 'directory_name' and 'directory_path' containing information
          about each directory within the project folder.

    expand_directory_df(self)
        Expands the directory DataFrame by adding additional columns related to the image analysis, such as sensor type,
        session ID, stimulation IDs, and stimulation frame numbers. This method parses directory names and CSV files
        within directories to extract this information.

        Steps Performed:
        1. Adds new columns for sensor type, session ID, stimulation IDs, and frame numbers with default values.
        2. Iterates through each directory, parsing the directory name for sensor type and session ID.
        3. Searches for specific CSV files within each directory and extracts stimulation details from them.

        Returns:
        - DataFrame: The updated pandas DataFrame with new columns for sensor type, session ID, stimulation IDs, and
          frame numbers populated based on the directory's content.

    """
    
    def __init__(self, project_folder):
        self.project_folder = project_folder
        self.directory_df = self.initialize_directory_df() 
        
    def initialize_directory_df(self):
        directories = [d for d in os.listdir(self.project_folder) if os.path.isdir(os.path.join(self.project_folder, d))]
        directory_data = [{'directory_name': d, 'directory_path': os.path.join(self.project_folder, d)} for d in directories]
        return pd.DataFrame(directory_data, columns=['directory_name', 'directory_path'])
    
    def expand_directory_df(self):
        # Add new columns with default empty lists
        self.directory_df['sensor_type'] = ''
        self.directory_df['session_id'] = ''
        self.directory_df['stimulation_ids'] = [[] for _ in range(len(self.directory_df))]
        self.directory_df['stimulation_frame_number'] = [[] for _ in range(len(self.directory_df))]

        for index, row in self.directory_df.iterrows():
            folder_name = row['directory_name']
            folder_path = row['directory_path']
            
            # Parse folder name for sensor type and session id
            parts = folder_name.split('_')
            sensor_type = 'gcamp8' if parts[0].startswith('g') else 'cablam'
            session_id = parts[0][1:] + parts[1]  # Assuming the first part is always the experiment ID

            # Update DataFrame with sensor_type and session_id
            self.directory_df.at[index, 'sensor_type'] = sensor_type
            self.directory_df.at[index, 'session_id'] = session_id

            # Check for CSV file ending in 'biolumi' or 'fluor'
            csv_filename = [f for f in os.listdir(folder_path) if (f.endswith('biolumi.csv') or f.endswith('fluor.csv'))]
            if csv_filename:
                csv_file_path = os.path.join(folder_path, csv_filename[0])
                df_csv = pd.read_csv(csv_file_path, header=None)
                stimulation_ids = df_csv.iloc[1].dropna().tolist()
                stimulation_frame_number = df_csv.iloc[0].dropna().tolist()

                # Update DataFrame with stimulation information
                self.directory_df.at[index, 'stimulation_ids'] = stimulation_ids
                self.directory_df.at[index, 'stimulation_frame_number'] = stimulation_frame_number

        return self.directory_df
        
    def max_projection_mean_values(self, tif_path):
        """
        Generates a maximum intensity projection based on the mean values of a multi-frame TIF file
        and saves it to a new subdirectory 'processed_data/processed_image_analysis_output'
        with a '_max_projection' suffix in the file name.

        Parameters:
        tif_path (str): Path to the multi-frame TIF file.

        Returns:
        str: Path to the saved maximum intensity projection image.
        """

        with Image.open(tif_path) as img:
            # Initialize a summing array with the shape of the first frame and float type for mean calculation
            sum_image = np.zeros((img.height, img.width), dtype=np.float32)

            # Sum up all frames
            for i in range(img.n_frames):
                img.seek(i)
                sum_image += np.array(img, dtype=np.float32)

            # Compute the mean image by dividing the sum by the number of frames
            mean_image = sum_image / img.n_frames
        
        # Define the new directory path
        processed_dir = os.path.join(os.path.dirname(tif_path), 'processed_data', 'processed_image_analysis_output')
        
        # Create the directory if it does not exist
        os.makedirs(processed_dir, exist_ok=True)
        
        # Create a new file path for the max projection image with the '_max_projection' suffix
        # The filename is extracted from tif_path and appended with '_max_projection.tif'
        file_name = os.path.basename(tif_path)
        max_proj_image_path = os.path.join(processed_dir, file_name.replace('.tif', '_max_projection.tif'))
       
        # Save the max projection image to the new file path
        Image.fromarray(mean_image).save(max_proj_image_path)

        # Return the path to the saved image
        return max_proj_image_path
    
    
    def analyze_all_sessions(self, function_to_apply):
        """
        Iterates over all session IDs in the directory DataFrame and applies the given function to each.

        Parameters:
        function_to_apply (callable): Function to be applied to each session. It should accept a session ID.

        Returns:
        dict: A dictionary with session_ids as keys and function return values as values.
        """
        results = {}
        for session_id in self.directory_df['session_id']:
            try:
                result = function_to_apply(session_id)
                results[session_id] = result
            except Exception as e:
                print(f"An error occurred while processing session {session_id}: {e}")
        return results
    
    def analyze_session_max_projection(self, session_id):
        """
        Wrapper function to apply max_projection_mean_values to a session's TIF file.

        Parameters:
        session_id (str): The session ID for which the TIF file will be processed.

        Returns:
        str: Path to the processed max projection TIFF file.
        """
        # analysis is an instance of ImageAnalysis
        tif_path = self.get_session_raw_data(session_id)
        if isinstance(tif_path, str) and tif_path.endswith('.tif'):
            return self.max_projection_mean_values(tif_path)
        else:
            return f"No valid TIF file found for session {session_id}"# Apply max_projection_mean_values to all sessions
        
    def get_session_raw_data(self, session_id):
        # Check if the session_id is in the 'session_id' column of the directory_df
        if session_id in self.directory_df['session_id'].tolist():
            # Find the directory path for the given session_id
            directory_path = self.directory_df[self.directory_df['session_id'] == session_id]['directory_path'].values[0]
            
            # Search for the .tif file within that directory
            for file_name in os.listdir(directory_path):
                if file_name.endswith('.tif'):
                    return os.path.join(directory_path, file_name)

            # If no .tif file is found in the directory
            return f"No .tif file found in the directory for session {session_id}."
        else:
            # If the session_id is not present in the DataFrame
            return f"Session ID {session_id} is not present in the directory DataFrame."
         
    def add_tiff_dimensions(self):
        """
        Analyzes the dimensions of TIF files in the directory DataFrame and adds this data as new columns.
        """
        # Ensure the DataFrame has the columns for dimensions
        if 'x_dim' not in self.directory_df.columns:
            self.directory_df['x_dim'] = None
            self.directory_df['y_dim'] = None
            self.directory_df['z_dim_frames'] = None

        # Iterate over each session_id and update the dimensions
        for index, row in self.directory_df.iterrows():
            tif_path = self.get_session_raw_data(row['session_id'])
            if isinstance(tif_path, str) and tif_path.endswith('.tif'):
                try:
                    with Image.open(tif_path) as img:
                        self.directory_df.at[index, 'x_dim'] = img.width
                        self.directory_df.at[index, 'y_dim'] = img.height
                        # For z-dimension, count the frames
                        img.seek(0)  # Ensure the pointer is at the beginning
                        frames = 0
                        while True:
                            try:
                                img.seek(img.tell() + 1)
                                frames += 1
                            except EOFError:
                                break
                        self.directory_df.at[index, 'z_dim_frames'] = frames
                except Exception as e:
                    print(f"Could not process TIF dimensions for session {row['session_id']}: {e}")
                                
    def analyze_all_calcium_signals(self):
        """
        Applies calcium signal extraction to all session_ids in the directory DataFrame and stores the results.
        """
        results = {}
        for session_id in self.directory_df['session_id']:
            # Ensure the ROI analysis has been done to get the labeled images
            roi_results = self.analyze_roi(session_id)
            # Check if analyze_roi returned a path to labeled images
            if isinstance(roi_results, tuple):
                # Extract calcium signals using the labeled ROI mask
                calcium_csv_path = self.extract_calcium_signals(session_id)
                results[session_id] = calcium_csv_path
            else:
                # If roi_results is an error message, pass it through
                results[session_id] = roi_results
                
            #results is a dictionary where each key is a session_id and the corresponding value is the path to the saved CSV file containing calcium signal data.
        return results
    
    
    def analyze_roi(self, session_id):
        """
        Analyzes ROI of the 'labels_postexport.tif' file for a given session and saves two results:
        one with labels and another without labels.I t also saves the labeled image data as numpy array for future use.
        """
        
        #### SETP 1: DEFINE PATHS ####
        # define the paths, including the directory where processed images will be saved (processed_dir) 
        # and the name of the TIF file that contains the ROI labels (consistent_file_name)
        processed_dir = 'processed_data/processed_image_analysis_output'
        consistent_file_name = 'labels_postexport.tif'
        output_suffix_with_labels = '_roi_analysis_with_labels.png'
        output_suffix_without_labels = '_roi_analysis_without_labels.png'

        #### STEP 2: RETRIEVE SESSION DATA ####
        # Retrieve the directory path from the DataFrame
        # looks up the session's directory path from a DataFrame (directory_df) using the provided session_id. 
        # If the session ID isn't found, it returns a message indicating no directory entry was found for that session.
        
        directory_entry = self.directory_df[self.directory_df['session_id'] == session_id]
        if directory_entry.empty:
            return f"No directory entry found for session {session_id}"

        #### STEP 3: VERIFY AND LOAD THE ROI TIF FILE ####
        # constructs the full path to the labels_postexport.tif file and checks if it exists. If it does, the file is opened and loaded. 
        # If the file is in RGB format, it's converted to grayscale using rgb2gray from skimage.color. 
        # This conversion is crucial for analyzing the image as a binary mask where non-white pixels are considered ROIs.
        directory_path = directory_entry['directory_path'].values[0]
        
        # Build the path to the postexport TIFF file
        tiff_file_path = os.path.join(directory_path, processed_dir, consistent_file_name)

        # Verify that the file exists
        if not os.path.exists(tiff_file_path):
            return f"File not found for session {session_id}"
        else:
            print(f"Analyzing session {session_id}...")
        
        #### STEP 4: CREATE AND SAVE THE BINARY MASK ####
        # k: The method then converts the grayscale image to a binary mask, identifying all non-white pixels as ROIs 
        # (pixels with value less than 1 after normalization are set to 1, and others to 0). 
        # This binary mask is labeled using label from skimage.measure, assigning a unique label to each connected component (ROI).
        
        # Load the image
        mask_image = Image.open(tiff_file_path)

        # Convert RGB image to grayscale if necessary
        if mask_image.mode == 'RGB':
            # Convert to grayscale using skimage's rgb2gray
            image_array = rgb2gray(np.array(mask_image))

        # Assuming that all non-white pixels are ROIs
        binary_mask = np.where(image_array < 1, 1, 0)  # Here, 1 corresponds to white in the normalized grayscale image

        # Label the regions
        labeled_image = label(binary_mask, connectivity=1)
        num_rois = np.max(labeled_image)
        
        # Save the labeled image data as a NumPy array file for future processing
        labeled_image_path = os.path.join(directory_path, processed_dir, f"{session_id}_labeled_image.npy")
        np.save(labeled_image_path, labeled_image)
        
        
        #### STEP 5: SAVE THE UNLABELED ROI IMAGE ####
        # Save Unlabeled ROI Image: The method saves a version of the labeled image without any annotations to a specified path (output_path_without_labels). 
        # This image is saved in the processed_image_analysis_output directory with a specific suffix to indicate it's the unlabeled version.
        
        # Save the image without labels
        output_path_without_labels = os.path.join(directory_path, processed_dir, session_id + output_suffix_without_labels)
        plt.imsave(output_path_without_labels, labeled_image, cmap='nipy_spectral')

        
        #### STEP 6: ANALYZE AND SAVE LABELED ROI IMAGE ####
        # Iterates through each detected region using regionprops, extracts the centroid, 
        # and annotates the image with the region's label. 
        # This annotated image is saved separately, indicating it includes ROI labels.
        
        # Analyze regions and save properties
        regions = regionprops(labeled_image)

        # Prepare to save the ROI analysis image with labels
        output_path_with_labels = os.path.join(directory_path, processed_dir, session_id + output_suffix_with_labels)
        
        fig, ax = plt.subplots()
        ax.imshow(labeled_image, cmap='nipy_spectral')
        ax.axis('off')

        # Annotate each ROI with its corresponding label (ID)
        for region in regions:
            # Get the coordinates of the centroid of the region
            y, x = region.centroid
            # Annotate the ROI ID at the centroid position
            ax.text(x, y, str(region.label), color='white', ha='center', va='center')

        plt.savefig(output_path_with_labels)
        plt.close()

        # Return the paths of the saved figures LABELED AND UNLABELED and number of ROIs
        return (output_path_with_labels, output_path_without_labels), num_rois
    
    def analyze_all_rois(self):
        """
        Applies ROI analysis to all sessions and saves the results.
        """
        results = {}
        for session_id in self.directory_df['session_id']:
            result = self.analyze_roi(session_id)
            results[session_id] = result
        return results
    
    def process_all_sessions(self, use_corrected_data=False):
        
        """
        This assumes the analyze_all_rois method has been previously run to generate the numpy files 
        and the corresponding images with and without labels for ROI per session.
        
        Process all sessions using either corrected or uncorrected calcium signal data.

        Parameters
        ----------
        use_corrected_data : bool, optional
            Flag indicating whether to use corrected calcium signals. Defaults to False, 
            indicating uncorrected data should be used.

        Returns
        -------
        dict
            A dictionary with processed data for all sessions, keyed by session ID.
        """
        
        all_data = {}
        for session_id in self.directory_df['session_id'].unique():
            stim_frame_numbers, roi_data, stimulation_ids = self.create_trial_locked_calcium_signals(session_id, use_corrected_data=use_corrected_data)
            if stim_frame_numbers and roi_data and stimulation_ids:  # Ensure data was returned
                all_data[session_id] = {
                    'stim_frame_numbers': stim_frame_numbers,
                    'roi_data': roi_data,
                    'stimulation_ids': stimulation_ids
                }
        return all_data
    
    def create_trial_locked_calcium_signals(self, session_id, use_corrected_data=False):
        """
        Generate trial-locked calcium signal data for a given session ID, allowing
        the choice between corrected and uncorrected data.
        
        Parameters
        ----------
        session_id : str
            The session ID for which to generate trial-locked signals.
        use_corrected_data : bool, optional
            Whether to use corrected calcium signal data. The default is False, which uses uncorrected data.
        
        Returns
        -------
        tuple
            A tuple containing the stimulation frame numbers, ROI data, and stimulation IDs.
        """

        processed_dir = 'processed_data/processed_image_analysis_output'
        calcium_csv_suffix = '_corrected_calcium_signals.csv' if use_corrected_data else '_calcium_signals.csv'

        directory_entry = self.directory_df[self.directory_df['session_id'] == session_id]

        if directory_entry.empty:
            print(f"No directory entry found for session {session_id}")
            return None, None, None

        directory_path = directory_entry['directory_path'].values[0]
        csv_path = os.path.join(directory_path, processed_dir, f"{session_id}{calcium_csv_suffix}")

        if not os.path.exists(csv_path):
            print(f"Calcium signals file not found for session {session_id} using {'corrected' if use_corrected_data else 'uncorrected'} data")
            return None, None, None

        calcium_signals_df = pd.read_csv(csv_path)
        stim_frame_numbers = directory_entry['stimulation_frame_number'].values[0]
        stimulation_ids = directory_entry['stimulation_ids'].values[0]

        pre_stim_frames = 10 # 10 frames before stimulation
        post_stim_frames = 100 # 100 frames after stimulation

        roi_data = {roi: {} for roi in calcium_signals_df.columns if 'ROI' in roi}

        for stim_id, stim_frame in zip(stimulation_ids, stim_frame_numbers):
            start_idx = max(stim_frame - pre_stim_frames, 0) 
            end_idx = min(stim_frame + post_stim_frames, len(calcium_signals_df))

            for roi in roi_data:
                trial = calcium_signals_df.loc[start_idx:end_idx, roi]
                roi_data[roi][(stim_id, stim_frame)] = trial.to_numpy()

        return stim_frame_numbers, roi_data, stimulation_ids
    

    

In [101]:
project_folder = '/Volumes/MannySSD/cablam_imaging/raw_data_template' #path to the folder containing the raw data to be analyzed (i.e. the folder containing the folders for each experiment)
analysis = ImageAnalysis(project_folder)
analysis.directory_df

,directory_name,directory_path
0,g12_12092023_estim_10hz_na_blk,/Volumes/MannySSD/cablam_imaging/raw_data_temp...
1,c21_12232023_estim_10hz_1xfz,/Volumes/MannySSD/cablam_imaging/raw_data_temp...


In [102]:
analysis.expand_directory_df()

,directory_name,directory_path,sensor_type,session_id,stimulation_ids,stimulation_frame_number
0,g12_12092023_estim_10hz_na_blk,/Volumes/MannySSD/cablam_imaging/raw_data_temp...,gcamp8,1212092023,"[12, 24, 36, 60, 120, 480]","[3579, 3780, 3982, 4183, 4385, 4587]"
1,c21_12232023_estim_10hz_1xfz,/Volumes/MannySSD/cablam_imaging/raw_data_temp...,cablam,2112232023,"[12, 24, 36, 60, 120, 240, 480, 960, 1920]","[3509, 3911, 4313, 4715, 5118, 5521, 5926, 633..."


At this point add your manual ROIs to the corresponding 'processed_image_analysis_output' directory. Ensure the .tiff files same matching xy pixel size to ensure the correct aligmen to the .tiff calcium movies . 

The code assumes the manual/ML generated masks are called 'labels_postexport.tif' and extracts this files itteratively for every unique session ID 


In [106]:
all_results = analysis.analyze_all_calcium_signals()

for session_id, csv_path in all_results.items():
    if isinstance(csv_path, str):
        print(f"Session {session_id} - Calcium signals saved at: {csv_path}")
    else:
        print(f"Session {session_id} - Error: {csv_path}")

AttributeError: 'ImageAnalysis' object has no attribute 'analyze_all_calcium_signals'

In [104]:
all_data = analysis.process_all_sessions(use_corrected_data=False)

Calcium signals file not found for session 1212092023 using uncorrected data
Calcium signals file not found for session 2112232023 using uncorrected data
